In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

path = r"Data\\"

In [2]:
from TransformData import transform_data
data_generator = transform_data(path = path + 'train.csv')
data = data_generator.get_data(train_or_analysis = 'train')

In [4]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

params = {'lambda': 3.792404253903758, 'alpha': 3.720406654006611, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.027569142191457396, 'max_depth': 7, 'random_state': 1, 'min_child_weight': 26}
x = data.drop('SalePrice', axis = 1)
y = data['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.3, random_state=1)
reg = XGBRegressor(eval_metric= 'rmse', n_estimators = 1433, **params)#, early_stopping_rounds = 50, **params)
reg.fit(x_train, y_train, 
        eval_set=[(x_train, y_train), (x_test, y_test)],
        verbose=100)
y_pred = reg.predict(x_test)

[0]	validation_0-rmse:11.22846	validation_1-rmse:11.20917
[100]	validation_0-rmse:0.79139	validation_1-rmse:0.78287
[200]	validation_0-rmse:0.19674	validation_1-rmse:0.20916
[300]	validation_0-rmse:0.15780	validation_1-rmse:0.17411
[400]	validation_0-rmse:0.14893	validation_1-rmse:0.16625
[500]	validation_0-rmse:0.14534	validation_1-rmse:0.16308
[600]	validation_0-rmse:0.14341	validation_1-rmse:0.16145
[700]	validation_0-rmse:0.14226	validation_1-rmse:0.16063
[800]	validation_0-rmse:0.14143	validation_1-rmse:0.16000
[900]	validation_0-rmse:0.14064	validation_1-rmse:0.15945
[1000]	validation_0-rmse:0.13993	validation_1-rmse:0.15893
[1100]	validation_0-rmse:0.13928	validation_1-rmse:0.15846
[1200]	validation_0-rmse:0.13860	validation_1-rmse:0.15795
[1300]	validation_0-rmse:0.13806	validation_1-rmse:0.15754
[1400]	validation_0-rmse:0.13773	validation_1-rmse:0.15736
[1432]	validation_0-rmse:0.13743	validation_1-rmse:0.15720


In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('---------test------------')
print("R²: %.2f" % r2)
print("MSE: %.2f" % mse)
print("RMSE: %.4f" % np.sqrt(mse))
print("MAPE: %.2f" % (mape * 100) + '%')

---------test------------
R²: 0.85
MSE: 0.02
RMSE: 0.1572
MAPE: 0.91%


In [5]:
tests = cross_val_score(reg, x_test, y_test, cv = 5, n_jobs = -1,scoring = 'neg_root_mean_squared_error')
np.mean(tests)

-0.1282965087515252

In [ ]:
def objective(trial):
    x = data.drop('SalePrice', axis = 1)
    y = data['SalePrice']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.3, random_state=1)
    param = {
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [1]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = XGBRegressor(**param, eval_metric= 'rmse')  
    model.fit(x_train,y_train,eval_set=[(x_train, y_train), (x_test, y_test)],verbose=False)
    preds = model.predict(x_test)
    tests = cross_val_score(model, x_test, y_test, cv = 5, n_jobs = -1,scoring = 'neg_root_mean_squared_error')
    return np.mean(tests)

In [ ]:
import optuna

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [7]:
importances = pd.DataFrame(columns = ['feature_importances'], data = reg.feature_importances_, index = reg.feature_names_in_).sort_values(by = 'feature_importances', ascending = False)


zero_importance= list(importances.loc[importances['feature_importances'] == 0].reset_index()['index'])
importances

,feature_importances
GarageCond_TA,0.143914
FullBath,0.121237
CentralAir_N,0.080009
ExterQual_TA,0.079460
OverallQual,0.079240
...,...
RoofMatl_Roll,0.000000
RoofMatl_Tar&Grv,0.000000
RoofMatl_WdShake,0.000000
RoofMatl_WdShngl,0.000000
